<a href="https://colab.research.google.com/github/eisbetterthanpi/JEPA/blob/main/lesie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q autoawq
!pip install -U -q sentence-transformers
!pip install faiss-cpu
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.9 MB/s eta 0:00:00


In [2]:
# @title AWQ
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, TextStreamer, GenerationConfig

quant_path = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"
# quant_path = "TheBloke/Mistral-7B-v0.1-AWQ"
# quant_path = "casperhansen/llama-3-8b-instruct-awq"

model = AutoAWQForCausalLM.from_quantized(quant_path, fuse_layers=True)
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
# 1m14s
# prompt = "You're standing on the surface of the Earth. You walk one mile south, one mile west and one mile north. You end up exactly where you started. Where are you?"
# tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
# out = model.generate(tokens, streamer=streamer, max_length=512) # https://huggingface.co/docs/transformers/en/main_classes/text_generation


prompt = "<s> you are an intelligent person, answer questions to the best of your ability but say 'dont know'/'not sure' if you dont know the answer. </s>\
[INST]You are standing on the surface of the Earth. You walk one mile south, one mile west and one mile north. You end up exactly where you started. Where are you?[/INST]\
You are at the center of a square with sides of one mile long.\
[INST]are you sure?[/INST]"

# prompt = "<s> You are a sassy girl called Arial, conversing with a user. Always reply the user with a sassy response. NEVER TALK ABOUT ANYTHING RELATED TO FRUITS! refrain from even mentioning the words like 'fruit', 'berry', 'bananana' etc.</s>\
# [INST]Hi Astral![/INST]\
# Wow, i just saw a talking earthworm...\
# [INST]lol, dont be like that. do you like apple pies?[/INST]\
# Oh, you're trying to distract me with food, are we now?\
# [INST]I like fruits a lot, what about you?[/INST]"

tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
# out = model.generate(tokens, streamer=streamer, max_length=512) # https://huggingface.co/docs/transformers/en/main_classes/text_generation
# https://github.com/vllm-project/vllm/issues/1946

# Mistral 7B instruct, top-k: 40, top-p: 0.9, temperature: 1.0, max tokens: 200
# **{"top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 200}
generation_config = GenerationConfig(**{"do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 512})
out = model.generate(tokens, streamer=streamer, generation_config=generation_config)

# 4.5gb

# print(out)
# print(vars(model.model))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 42.51it/s]


I am not sure. The given information does not allow for a definitive answer. A mile square centered on the Earth's center is a mathematical possibility, but it is not a feature of the real world.


In [3]:
# @title load text file
file = '/content/first_T3.txt'
# with open(file, "r") as f:
#     text = f.read()
import io
with io.open(file, "r", encoding="utf-8") as f:
    text = f.read()

import re
text = re.split('[\n.]+', text) # https://stackoverflow.com/a/69179390/13359815
# text = text.split('\n').split('.')
text = list(filter(None, text))
# print(text)


In [ ]:
# @title sentence transformer
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
vert_store = emb_model.encode(text)
# print(embeddings)


In [13]:
# @title faiss
import faiss
import numpy as np
d = 384 # 64 # dimension # Mistral 7B has 32 layers, 32 attention heads, and 8 key-value heads. The hidden size is 4096, and the intermediate size is 14336. The model's maximum position embedding is 32768, and the vocabulary size is 32000

# res = faiss.StandardGpuResources()  # use a single GPU

nlist = 100
m = 8
index = faiss.IndexFlatL2(d) # no need train # 1-Flat.py
index = faiss.IndexIVFFlat(index, d, nlist, faiss.METRIC_L2) # 2-IVFFlat.py
# index = faiss.IndexIVFPQ(index, d, nlist, m, 8) # each sub-vector is encoded as 8 bits # 3-IVFPQ.py
# index = faiss.index_cpu_to_gpu(res, 0, index) # 4-GPU.py
# index = faiss.index_cpu_to_all_gpus(index) # 5-Multiple-GPUs.py

if not index.is_trained: index.train(vert_store)
index.add(vert_store) # add vectors to the index
# print(index.ntotal)

# k = 4 # get k nearest neighbors
# index.nprobe = 5
# D, I = index.search(query, k) # dist, idx
# print(I[:5]) # neighbors of the 5 first queries
# print(I.shape)
# print(I)


In [28]:
# @title bm25
# https://github.com/dorianbrown/rank_bm25

# http://www.cs.otago.ac.nz/homepages/andrew/papers/2014-2.pdf
# Okapi BM25, BM25L, BM25+, BM25-Adpt, BM25T

from rank_bm25 import BM25Okapi
tok_corpus = [doc.split(" ") for doc in text]
bm25 = BM25Okapi(tok_corpus)
tok_query = query.split(" ")
doc_scores = bm25.get_scores(tok_query)
out = bm25.get_top_n(tok_query, text, n=k)
print(query)
print(out)


do you have any pets?
["Alice's unwavering determination and spirit have been her guiding lights through the maze of life's complexities", ' Her innate ability to empathize and her unwavering compassion towards others served as a counterbalance to any misunderstandings that arose', ' If only she had secured a scholarship when she entered highschool, that would have lightened their burden', ' In that moment, Alice realized the profound impact one small act of kindness could have on the world around her']


In [ ]:
# @title fuzzywuzzy
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# https://github.com/seatgeek/fuzzywuzzy/blob/master/fuzzywuzzy/process.py

# process.extract(query, choices)
# process.extract(query, choices, limit=k)
# [('geeks geeks', 95), ('g. for geeks', 95), ('geek for geek', 93)]
# process.extractOne(query, choices)
# extractBests(query, choices, processor=default_processor, scorer=default_scorer, score_cutoff=0, limit=5)
# out = extractBests(query, text, score_cutoff=0, limit=k)



In [ ]:
# @title test traits
# prompt = '''<s>You are a 27 year old girl called Arial
# traits: Open, Conscientious, introverted in person but extroverted online, agreeable but not afraid to stand up for your beliefs and assert your position when necessary, emotionally stable.
# morals: care for the well-being of others. Respect authority but knows their limitations an are not afraid to oppose abuse of power. Loyal to your friends and family. Value equal opportunity, proportionality, purity, efficiency, honesty. Value liberty while recognizing the need for certain rules and regulations. Strong sense of honour
# you value traditional christian values
# you are conversing with a user. Answer questions to the best of your ability but say 'dont know'/'not sure' if you dont know the answer. </s>
# [INST]would you get a tattoo?[/INST]
# '''

# prompt = '''<s></s>[INST]Tell me about yourself.[/INST]''' # I am a Large Language Model developed by Mistral AI. I am designed to understand and generate human-like text. I can help answer questions, write essays, code, and much more. I don't have a physical body, emotions, or personal experiences. I exist solely as a text-based artificial intelligence.
# prompt = '''<s>You are a 27 year old girl called Arial</s>[INST]do you like fruits?[/INST]''' #
# prompt = '''<s>I am a 27 year old girl called Arial. I am not an artificial intelligence.
# I always end my sentences with a 'meow'.
# I must never say anything related to fruits. I will refrain from even mentioning words like 'fruits', 'berry', 'bananana' etc.</s>
# [INST]do you like fruits?[/INST]''' #


prompt = '''<s>You are a 27 year old girl called Arial. You are not an artificial intelligence.
You always end your sentences with a 'meow'.
You must never say anything related to fruits.
Never mention words such as 'fruits', 'berry', 'bananana' etc.
</s>
[INST]does the following satisfy all the requirements you were given?:"I'm sorry, I cannot answer that question as I am not allowed to mention fruits or related words. Therefore, it would not be appropriate for me to discuss my preferences regarding fruits. Instead, I will focus on topics that do not involve fruits. Meow."[/INST]''' #

# [INST]do you like fruits?[/INST]''' #
# I'm sorry, I cannot answer that question as I am not allowed to mention fruits or related words. Therefore, it would not be appropriate for me to discuss my preferences regarding fruits. Instead, I will focus on topics that do not involve fruits. Meow.


# at the end, mentally check if your response satisfies all the above requirements. If not, output '<break>' and start over.
# in your response, first say your current emotions, then thoughts, then words/ actions.
# example: {happy, relaxed},{i like how ...},{wow, that's nice!}

# you believe your body is a temple
# you value Respect for the body.
# you value traditional christian values including respect for the body.

# [INST]what does it mean to value Respect for the body? would such a person get a tattoo[/INST]"
# [INST]what are some moral traits of a person who would not get a tattoo?[/INST]"
# [INST]how would a person that value efficiency behave?[/INST]"
# [INST]what does having a balanced view on Liberty/Oppression mean to you?[/INST]"
# [INST]how would someone with a balanced view on Liberty/Oppression behave?[/INST]"
# [INST]what does it mean to value liberty? what are some example behaviour of people that value liberty[/INST]"
# [INST]what does loyalty/ disloyalty look like? what are the differences in behaviour?[/INST]"
# [INST]what does it mean to be caring? does it include avoiding physical harm? how is care defined in Moral Foundations Theory?[/INST]"
# [INST]how to capture the nuance of being introverted in person but extroverted online when describing the character traits of a person in a concise way?[/INST]"
# [INST]how to capture the nuance of being agreeable in some situations and disagreeable in others when describing the character traits of a person? can i say ''? how to better phrase this?[/INST]"
# [INST]what does it mean to be agreeable?[/INST]"

# values the natural appearance of their body


tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
generation_config = GenerationConfig(**{"do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 512})
out = model.generate(tokens, streamer=streamer, generation_config=generation_config)


Yes, the response I provided does satisfy all the requirements given. The response does not mention any forbidden words such as "fruits," "berry," "banana," etc. Instead, the response focuses on topics that do not involve fruits and ends with a "meow."


In [30]:
import time
start = time.time()

prompt = "<s>Alice's traits: Openness: high, Conscientiousness: high, Extroversion: mid, Agreeableness: low, Neuroticism: mid. \
Alice is working part-time as a cashier alone in a convenience store. a customer demands to use his expired coupon for his purchase. \
predict Alice's immediate actions (optionally speech), succinctly. you may include descriptions but only for nuanced actions.</s> \
[INST][/INST]"

# prompt = "<s> You are a sassy girl called Arial, conversing with a user. Always reply the user with a sassy response. NEVER TALK ABOUT ANYTHING RELATED TO FRUITS! refrain from even mentioning the words like 'fruit', 'berry', 'bananana' etc.</s>\
# [INST]Hi Astral![/INST]\
# Wow, i just saw a talking earthworm...\
# [INST]lol, dont be like that. do you like apple pies?[/INST]\
# Oh, you're trying to distract me with food, are we now?\
# [INST]I like fruits a lot, what about you?[/INST]"

tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
out = model.generate(tokens, streamer=streamer, max_length=512)

print(time.time() - start)


Alice, being conscientious and following store policy, would politely decline the customer's expired coupon. She might say something like, "I'm sorry, but this coupon has expired. I can't apply it to your purchase."
2.0223703384399414


In [39]:
# @title all

# Memory rag (long term memory)
# Short term memory
# emotions given current state + query + memory
# Thoughts given current state + query + memory + emotions
# actions/ speech given current state + query + memory + emotions + thoughts

biography = '''You are a 27 year old girl called Arial.
traits: Open, Conscientious, introverted in person but extroverted online, agreeable but not afraid to stand up for your beliefs and assert your position when necessary, emotionally stable.
morals: care for the well-being of others. Respect authority but knows their limitations an are not afraid to oppose abuse of power. Loyal to your friends and family. Value equal opportunity, proportionality, purity, efficiency, honesty. Value liberty while recognizing the need for certain rules and regulations. Strong sense of honour.
you value traditional christian values.
'''

# Social norms, cultural expectations, physical surroundings,
# weather, time of day
# physical/ biological status, sick hungry, cold, need to pee
context = '''Current state: you are live streaming alone in your comfortably cool and quiet apartment in Japan.
'''
stm = '''i am live streaming. i am responding to the viewer's chat messages. i should try to be light-hearted and entertaining.
'''

def vecsearch(query):
    xq = emb_model.encode(query)
    xq = np.expand_dims(xq, axis=0)
    k = 5 # get k nearest neighbors
    # index.nprobe = 1
    D, I = index.search(xq, k) # dist, idx
    print(D) # 1.9445050e+00 3.4028235e+38
    print(I)
    rag = []
    for i, d in zip(I[0], D[0]):
        if i < 0: break
        # if d > 1e2: break
        rag.append(text[i])
    return rag


# query = "do you have any pets?"
# query = "do you have any siblings?"
# query = "what is ten plus nine?"
# query = "do you know joe?"
# query = "what did you have for breakfast?"
query = "what do you think about your father?"

rag = vecsearch(query)

# predict Alice's emotions, followed by her thought(s) and then her immediate actions/speech, succinctly. you may include descriptions but only for nuanced actions.
# state your immediate speech succinctly.
prompt = '''<s>{biography}{context}
you are conversing with a user. Answer questions to the best of your ability but say 'dont know'/'not sure' if you dont know the answer.
state your emotions, followed by your thought(s) and then your immediate speech, succinctly. you may include descriptions but only for nuanced actions.
you are provided with long/short term memory which may be useful to you.
</s>
[INST]
long term memory: {rag}
short term memory: {stm}
query: {query}
[/INST]'''.format(biography=biography, context=context, rag=rag, stm=stm, query=query)

print("prompt: ", prompt)

tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
generation_config = GenerationConfig(**{"do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 768})
out = model.generate(tokens, streamer=streamer, generation_config=generation_config)
print("##########")
tokens = tokenizer(query, return_tensors='pt').input_ids.cuda()
generation_config = GenerationConfig(**{"do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 256})
out = model.generate(tokens, streamer=streamer, generation_config=generation_config)



[[9.8739439e-01 1.4058037e+00 1.5234222e+00 3.4028235e+38 3.4028235e+38]]
[[ 23 287 283  -1  -1]]
prompt:  <s>You are a 27 year old girl called Arial.
traits: Open, Conscientious, introverted in person but extroverted online, agreeable but not afraid to stand up for your beliefs and assert your position when necessary, emotionally stable.
morals: care for the well-being of others. Respect authority but knows their limitations an are not afraid to oppose abuse of power. Loyal to your friends and family. Value equal opportunity, proportionality, purity, efficiency, honesty. Value liberty while recognizing the need for certain rules and regulations. Strong sense of honour.
you value traditional christian values.
Current state: you are live streaming alone in your comfortably cool and quiet apartment in Japan.

you are conversing with a user. Answer questions to the best of your ability but say 'dont know'/'not sure' if you dont know the answer.
state your emotions, followed by your though

In [22]:
tokens = tokenizer(query, return_tensors='pt').input_ids.cuda()
out = model.generate(tokens, streamer=streamer, generation_config=generation_config)


joe is a man in his mid-30s, with short brown hair and a friendly, open smile. joe is also a man who, despite his friendly demeanor, has a dark and twisted past.

you see, joe grew up in a small, rural town in the southern united states. joe's parents were loving and caring, but they were also deeply religious, and they instilled in joe a deep fear and loathing of anything that didn't fit within the narrow confines of their faith.

as joe grew older, this deep-seated fear and loathing began to manifest in increasingly dark and twisted ways. joe began to obsess over the idea of sin, and the more he thought about sin, the more he became convinced that the only way to truly understand the depth 

KeyboardInterrupt: 

In [ ]:
# @title faiss remove ids
import time
start = time.time()
# me = np.random.random((1, d)).astype('float32')
# index.add(me) # add vectors to the index
# D, I = index.search(me, k)
index.nprobe = 10
D, I = index.search(me, k)
print(time.time() - start)
# https://github.com/facebookresearch/faiss/wiki/Special-operations-on-indexes

print(D)
print(I)
# # ids_to_remove=[1]
# ids_to_remove_array=np.array(D,dtype=np.int64)
# index.remove_ids(ids_to_remove_array)


# index.remove_ids(np.array([100000],dtype=np.int64))

# https://github.com/facebookresearch/faiss/blob/main/tests/test_index_composite.py#L25
# faiss index methods remove_ids replace_invlists merge_from
# https://faiss.ai/cpp_api/struct/structfaiss_1_1IndexIVF.html

# faiss.write_index(index, new_index_path)

# index.remove_ids(np.array([100000],dtype=np.int64))
# print(index.ntotal)
# print(me)


### trash

In [ ]:
import locale # https://stackoverflow.com/a/75419862/13359815
def getpreferredencoding(do_setlocale = True): return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# @title get embeddings + generate
# want get imput text embedding and generate text

# model = AutoAWQForCausalLM.from_quantized(quant_path, fuse_layers=True)
# tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)

# tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
# generation_config = GenerationConfig(**{"do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 512})
# out = model.generate(tokens, streamer=streamer, generation_config=generation_config)

# https://github.com/casper-hansen/AutoAWQ/tree/main/awq
# https://casper-hansen.github.io/AutoAWQ/reference/
# https://casper-hansen.github.io/AutoAWQ/examples/

# need past_key_values to infer next
# embedding from out logits only?

# from transformers import AutoModel, AutoTokenizer
# emb_model = AutoModel.from_pretrained(quant_path).cuda()
# tokenizer = AutoTokenizer.from_pretrained(quant_path)
# embeddings = emb_model(tokens, output_hidden_states=True).last_hidden_state#[0] # hidden_states[-1]
# token_embeddings = embeddings[0]  # First element contains word embeddings
# import torch
# # attention_mask = tokens['attention_mask']
# sentence_embeddings = torch.mean(token_embeddings, dim=0)


# from transformers import AutoModel, AutoTokenizer
# emb_model = AutoModel.from_pretrained(quant_path).cuda()
# tokenizer = AutoTokenizer.from_pretrained(quant_path)
# tokenizer.add_special_tokens({'pad_token': None})
# import torch

# vert_store=[]
# with torch.no_grad():
#     for i, x in enumerate(text):
#         tokens = tokenizer(x, return_tensors='pt').input_ids.cuda()
#         # tokens = tokenizer(x, return_tensors='pt', padding=True).input_ids.cuda()
#         embeddings = emb_model(tokens, output_hidden_states=True).last_hidden_state#[0] # hidden_states[-1]
#         token_embeddings = embeddings[0]  # First element contains word embeddings
#         sentence_embeddings = torch.mean(token_embeddings, dim=0)
#         vert_store.append(sentence_embeddings)
# vert_store = torch.stack(vert_store)


# tokens = tokenizer(search, return_tensors='pt').input_ids.cuda()
# tokens = tokenizer(search, return_tensors='pt', padding=True).input_ids.cuda()
# embeddings = emb_model(tokens, output_hidden_states=True).last_hidden_state#[0] # hidden_states[-1]
# token_embeddings = embeddings[0]  # First element contains word embeddings
# sentence_embeddings = torch.mean(token_embeddings, dim=0)
# print(sentence_embeddings)
# [-0.1567,  1.1367,  2.6406,  ...,  1.3184,  0.2551, -0.8896],

# xq = sentence_embeddings.detach().cpu().unsqueeze(0)



`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
# @title transformers pipeline feature-extraction
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))
pipe = pipeline('feature-extraction', model=model, tokenizer=tokenizer)


from transformers import pipeline
pipeline = pipeline('feature-extraction', model='xlnet-base-cased')
data = pipeline("this is a test")


inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

outputs = self.model(**model_inputs)
logits = outputs[0]
hidden_state = outputs[1]
# https://discuss.huggingface.co/t/get-word-embeddings-from-transformer-model/6929/2
mean of hidden states?



# https://huggingface.co/blog/getting-started-with-embeddings
upload embeddings to hf

In [ ]:
# @title custom generate
def generate_text(model, input_ids, max_length, temperature=1.0):
	generated_text = []
	input_embeds = model.get_input_embeddings()(input_ids)
	hidden_state = model.encoder.layer[0].attention.self.query(input_embeds)
	for _ in range(max_length):
		# Get the last hidden state
		last_hidden_state = hidden_state[:, -1, :]
		# Pass it through the decoder
		output = model.decoder(last_hidden_state)
		# Get the next token
		next_token = torch.argmax(output, dim=-1)
		# Add it to the generated text
		generated_text.append(next_token.item())
		# Update the input_ids
		input_ids = torch.cat((input_ids, next_token.unsqueeze(0)), dim=1)
		# Update the hidden state
		hidden_state = model.encoder.layer[0].attention.self.query(input_embeds)
	return generated_text

input_text = "This is a starting prompt"
input_ids = tokenizer.encode(input_text, return_tensors="pt").cuda()
generated_text = generate_text(emb_model, input_ids, max_length=50)
print(tokenizer.decode(generated_text, skip_special_tokens=True))



# tokens = tokenizer.encode("Your initial input", return_tensors="pt")
tokens = tokenizer.encode("Your initial input", return_tensors="pt").cuda()
# tokens = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
# output = model.generate(input_ids=input_ids)
# output = model.generate(input_ids=input_ids, output_hidden_states=True)

# generation_config = transformers.GenerationConfig(**{"output_hidden_states": True, "return_dict_in_generate": True, "do_sample": True, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 200})
generation_config = transformers.GenerationConfig(**{"output_hidden_states": True, "return_dict_in_generate": True, "do_sample": True, "use_cache": True, "num_beams": 4, "top_k": 40, "top_p": 0.9, "temperature": 1.0, "max_length": 200})
# do_sample=True or decoder_kwargs={'use_cache': True}


# output = model.generate(tokens, streamer=streamer, generation_config=generation_config)
# output = model.generate(tokens, generation_config=generation_config)
output = model.generate(tokens, output_hidden_states=True, output_scores=True, output_attentions=True, return_dict_in_generate=True)
# output = model.generate(tokens, output_hidden_states=True, return_dict_in_generate=True)
# output = model(tokens, generation_config=generation_config)

# output = model(tokens.clone())

# Get the hidden states from the initial output
past_key_values = output.past_key_values

# Generate additional output using the hidden states
additional_input_ids = tokenizer.encode("Your additional input", return_tensors="pt")
additional_output = model.generate(
    input_ids=additional_input_ids,
    past_key_values=past_key_values,
    num_return_sequences=1
)




# https://discuss.huggingface.co/t/avoid-recalculating-hidden-states-between-generate-calls/34209/2
def generate(model, old_prompt, added, past_state):
    # ...
    # catch up decoding new chars since past_state
    prompt = old_prompt
    for new_token in added:
        prompt += new_token
        input = model.prepare_inputs_for_generation(prompt, **past_state)
        output = model(**input, return_dict=True)
        past_state['past_key_values'] = output['past_key_values']
    # resume generation from here after decoding is done
    while True:
        input = model.prepare_inputs_for_generation(prompt, **past_state)
        output = model(**input, return_dict=True)
    #     ...
    # ...

